In [507]:
# pip install --upgrade spacy

Note: you may need to restart the kernel to use updated packages.


In [508]:
# !python -m spacy download en_core_web_md

     ---------------------------------------- 33.5/33.5 MB 6.0 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [509]:
# pip install podium-nlp

Note: you may need to restart the kernel to use updated packages.


In [1]:
# pip install hdbscan

In [3]:
# pip install bertopic

  Using cached numpy-1.21.6-cp39-cp39-win_amd64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tmtoolkit 0.11.2 requires numpy>=1.22.0, but you have numpy 1.21.6 which is incompatible.


In [511]:
# pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [1]:
from tqdm import tqdm
from gensim.utils import simple_preprocess
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
import re
import emoji
import copy

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# %cd drive/MyDrive/datasets/

# Preprocessing

In [2]:
df = pd.read_csv("train/SemEval2018-T3-train-taskA.txt", sep='\t', lineterminator='\n', encoding='utf-8')
df_test = pd.read_csv("goldtest_TaskA/SemEval2018-T3_gold_test_taskA_emoji.txt", sep='\t', lineterminator='\n', encoding='utf-8')
df_replace = pd.read_csv("test_TaskA/SemEval2018-T3_input_test_taskA.txt", sep='\t', lineterminator='\n', encoding='utf-8')

In [3]:
df.rename({'Tweet text\r': 'Tweet text'}, inplace=True, axis=1)
df_test.rename({'Tweet text\r': 'Tweet text'}, inplace=True, axis=1)
df_replace.rename({'Tweet text\r': 'Tweet text'}, inplace=True, axis=1)

In [4]:
df['Tweet text'].apply(emoji.demojize)

0       Sweet United Nations video. Just in time for C...
1       @mrdahl87 We are rumored to have talked to Erv...
2       Hey there! Nice to see you Minnesota/ND Winter...
3                   3 episodes left I'm dying over here\r
4       I can't breathe! was chosen as the most notabl...
                              ...                        
3812    @banditelli regarding what the PSU president d...
3813    @banditelli But still bothers me that I see no...
3814    well now that i've listened to all of into the...
3815    Hummingbirds #Are  #Experts #at #Hovering #Aft...
3816    Only thing missing now is a session at the gym...
Name: Tweet text, Length: 3817, dtype: object

In [38]:
#df_test['Tweet text'] = df_test['Tweet text'].apply(emoji.demojize)

## Getting clean text

In [5]:
def preprocess_words(s):
    '''
    Removes tags, emojis, links, smiley faces, hashtag signs, 
    stopwords and changes the case to lower.
    '''
    ret_list = []

    smiley_regex = r'([\:\;\=][\(\)PDO\/\\\]\[]+)+'
    
    is_tag = lambda w: w.startswith('@')
    is_vertical_line = lambda w: w.startswith('|')
    is_emoji = lambda w: w != ':' and w.startswith(':') and w.endswith(':')
    remove_emoji = lambda w: w[:w.index(':')] + w[w.rindex(':')+1:] if ':' in w and w.index(':') != w.rindex(':') else w
    is_link = lambda w: w.startswith("http") or w.startswith("https")
    is_hashtag = lambda w: w.startswith("#")
    is_smiley = lambda w: re.match(smiley_regex, w)
    
    for i, s_i in enumerate(s):
        w_arr = s_i.split()
        w2 = []
        for w in w_arr:
            if is_tag(w) or is_emoji(w) or is_link(w) or is_vertical_line(w):
                continue
            
            elif is_hashtag(w):
                w_tmp = w.replace('#', '')
                if w_tmp != '':
                    lower_append(w_tmp, w2)
            
            elif is_smiley(w):
                w_tmp = re.sub(smiley_regex, '', w)
                if w_tmp != '':
                    lower_append(w_tmp, w2)
            
            else:
                if w != '':
                    w.replace('#', '')
                    w.replace('|', '')
                    w.replace('_', '')
                    lower_append(w, w2)

        ret_list.append(' '.join(w2))
    return ret_list

def lower_append(w, l):
    l.append(w.lower())

df['clean_text'] = df[['Tweet text']].apply(preprocess_words)
df_test['clean_text'] = df_test[['Tweet text']].apply(preprocess_words)

In [6]:
print(df.shape)
df.head(15)

(3817, 4)


,Tweet index,Label,Tweet text,clean_text
0,1,1,Sweet United Nations video. Just in time for C...,sweet united nations video. just in time for c...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,we are rumored to have talked to erv's agent.....
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,hey there! nice to see you minnesota/nd winter...
3,4,0,3 episodes left I'm dying over here\r,3 episodes left i'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...,i can't breathe! was chosen as the most notabl...
5,6,0,You're never too old for Footie Pajamas. http:...,you're never too old for footie pajamas.
6,7,1,Nothing makes me happier then getting on the h...,nothing makes me happier then getting on the h...
7,8,0,4:30 an opening my first beer now gonna be a l...,4:30 an opening my first beer now gonna be a l...
8,9,0,@Adam_Klug do you think you would support a gu...,do you think you would support a guy who knock...
9,10,0,@samcguigan544 You are not allowed to open tha...,you are not allowed to open that until christm...


In [7]:
def simple_preprocessing(s):
    '''Lowercases, tokenizes, de-accents, removes words shorter than 3 and longer than 14 characters'''
    return [' '.join(simple_preprocess(s_i)) for s_i in s]

df['clean_text'] = df[['clean_text']].apply(simple_preprocessing)
df_test['clean_text'] = df_test[['clean_text']].apply(simple_preprocessing)
# tweet_text_train = tweet_text_train.apply(simple_preprocessing)
# tweet_text_test = tweet_text_test.apply(simple_preprocessing)

In [8]:
print(df.shape)
df.head(15)

(3817, 4)


,Tweet index,Label,Tweet text,clean_text
0,1,1,Sweet United Nations video. Just in time for C...,sweet united nations video just in time for ch...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,we are rumored to have talked to erv agent and...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,hey there nice to see you minnesota nd winter ...
3,4,0,3 episodes left I'm dying over here\r,episodes left dying over here
4,5,1,I can't breathe! was chosen as the most notabl...,can breathe was chosen as the most notable quo...
5,6,0,You're never too old for Footie Pajamas. http:...,you re never too old for footie pajamas
6,7,1,Nothing makes me happier then getting on the h...,nothing makes me happier then getting on the h...
7,8,0,4:30 an opening my first beer now gonna be a l...,an opening my first beer now gonna be long nig...
8,9,0,@Adam_Klug do you think you would support a gu...,do you think you would support guy who knocked...
9,10,0,@samcguigan544 You are not allowed to open tha...,you are not allowed to open that until christm...


In [9]:
def remove_tuple_characters(s):
    return [re.sub(r'(.)\1{2,}', r'\1', w) for w in s]

df['clean_text'] = df[['clean_text']].apply(remove_tuple_characters)
df_test['clean_text'] = df_test[['clean_text']].apply(remove_tuple_characters)
# tweet_text_train = tweet_text_train.apply(remove_tuple_characters)
# tweet_text_test = tweet_text_test.apply(remove_tuple_characters)

print(df.shape)
df.head(15)

(3817, 4)


,Tweet index,Label,Tweet text,clean_text
0,1,1,Sweet United Nations video. Just in time for C...,sweet united nations video just in time for ch...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,we are rumored to have talked to erv agent and...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,hey there nice to see you minnesota nd winter ...
3,4,0,3 episodes left I'm dying over here\r,episodes left dying over here
4,5,1,I can't breathe! was chosen as the most notabl...,can breathe was chosen as the most notable quo...
5,6,0,You're never too old for Footie Pajamas. http:...,you re never too old for footie pajamas
6,7,1,Nothing makes me happier then getting on the h...,nothing makes me happier then getting on the h...
7,8,0,4:30 an opening my first beer now gonna be a l...,an opening my first beer now gonna be long nig...
8,9,0,@Adam_Klug do you think you would support a gu...,do you think you would support guy who knocked...
9,10,0,@samcguigan544 You are not allowed to open tha...,you are not allowed to open that until christm...


In [10]:
nlp = spacy.load("en_core_web_md")

In [11]:
def lemmatize(s):
    '''Lemmatizes the words in the sentences and returns them if theyre not stopwords or punctuation'''
    return [[w.lemma_.lower() for w in nlp(s_i) if w.lemma_.lower() not in nlp.Defaults.stop_words and not w.is_punct] for s_i in s]

df['lemmas'] = df[['clean_text']].apply(lemmatize)
df_test['lemmas'] = df_test[['clean_text']].apply(lemmatize)
# tweet_text_train = tweet_text_train.apply(lemmatize)
# tweet_text_test = tweet_text_test.apply(lemmatize)

In [12]:
df.head(15)

,Tweet index,Label,Tweet text,clean_text,lemmas
0,1,1,Sweet United Nations video. Just in time for C...,sweet united nations video just in time for ch...,"[sweet, united, nations, video, time, christma..."
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,we are rumored to have talked to erv agent and...,"[rumor, talk, erv, agent, angel, ask, ed, esco..."
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,hey there nice to see you minnesota nd winter ...,"[hey, nice, minnesota, nd, winter, weather]"
3,4,0,3 episodes left I'm dying over here\r,episodes left dying over here,"[episode, leave, die]"
4,5,1,I can't breathe! was chosen as the most notabl...,can breathe was chosen as the most notable quo...,"[breathe, choose, notable, quote, year, annual..."
5,6,0,You're never too old for Footie Pajamas. http:...,you re never too old for footie pajamas,"[old, footie, pajama]"
6,7,1,Nothing makes me happier then getting on the h...,nothing makes me happier then getting on the h...,"[happy, highway, break, light, light, like, ch..."
7,8,0,4:30 an opening my first beer now gonna be a l...,an opening my first beer now gonna be long nig...,"[opening, beer, going, long, night, day]"
8,9,0,@Adam_Klug do you think you would support a gu...,do you think you would support guy who knocked...,"[think, support, guy, knock, daughter, rice, d..."
9,10,0,@samcguigan544 You are not allowed to open tha...,you are not allowed to open that until christm...,"[allow, open, christmas, day]"


In [13]:
def word_counter(s):
    splitted = s.split()
    newlist = [x for x in splitted if not x.startswith(("@", "#", "http"))]
    return len(newlist)

def char_counter(s):
    s_new = copy.copy(s)
    s_new = s_new.replace(' ', '')
    return len(s_new)

def tag_counter(s):
    splitted = s.split()
    newlist = [x for x in splitted if x.startswith("@")]
    return len(newlist)

def hashtag_counter(s):
    splitted = s.split()
    newlist = [x for x in splitted if x.startswith("#")]
    return len(newlist)

def has_emoji(s):
    ## potreban upgrade, ne pronalazi sve emoji-e
    splitted = s.split()
    newlist = [x for x in splitted if x != ":" and x.startswith(":") and x.endswith(":")]
    return len(newlist)

# def clean_text(s):
#     splitted = s.split()
#     newlist = [x for x in splitted if not x.startswith((":", "@", "#"))]
#     return ' '.join(newlist)

### dodani featuresi ###
def link_counter(s):
    splitted = s.split()
    newlist = [x for x in splitted if x.startswith(('http:', 'https:'))]
    return len(newlist)

def smiley_counter(s):
    splitted = s.split()
    newlist = [x for x in splitted if re.match(r'([\:\;\=][\(\)PDO\/\\\]\[]+)+', x)]
    return len(newlist)

# def exclamation_mark_counter(s):
#     return s.count('!')

def mark_counter(s):
    return (s.count('!') + s.count('?') + s.count('...'))



In [14]:
df['word_count'] = df['clean_text'].apply(word_counter)
df['char_count'] = df['clean_text'].apply(char_counter)
df['tag_count'] = df['Tweet text'].apply(tag_counter)
df['hashtag_count'] = df['Tweet text'].apply(hashtag_counter)
df['link_count'] = df['Tweet text'].apply(link_counter)
df['smiley_count'] = df['Tweet text'].apply(smiley_counter)
df['mark_count'] = df['Tweet text'].apply(mark_counter)
df['has_emoji'] = df['Tweet text'].apply(has_emoji)

In [15]:
df_test['word_count'] = df_test['clean_text'].apply(word_counter)
df_test['char_count'] = df_test['clean_text'].apply(char_counter)
df_test['tag_count'] = df_test['Tweet text'].apply(tag_counter)
df_test['hashtag_count'] = df_test['Tweet text'].apply(hashtag_counter)
df_test['link_count'] = df_test['Tweet text'].apply(link_counter)
df_test['smiley_count'] = df_test['Tweet text'].apply(smiley_counter)
df_test['mark_count'] = df_test['Tweet text'].apply(mark_counter)
df_test['has_emoji'] = df_test['Tweet text'].apply(has_emoji)

In [16]:
df

,Tweet index,Label,Tweet text,clean_text,lemmas,word_count,char_count,tag_count,hashtag_count,link_count,smiley_count,mark_count,has_emoji
0,1,1,Sweet United Nations video. Just in time for C...,sweet united nations video just in time for ch...,"[sweet, united, nations, video, time, christma...",11,62,0,2,1,0,0,0
1,2,1,@mrdahl87 We are rumored to have talked to Erv...,we are rumored to have talked to erv agent and...,"[rumor, talk, erv, agent, angel, ask, ed, esco...",19,82,1,0,0,1,2,0
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...,hey there nice to see you minnesota nd winter ...,"[hey, nice, minnesota, nd, winter, weather]",10,44,0,0,0,0,1,0
3,4,0,3 episodes left I'm dying over here\r,episodes left dying over here,"[episode, leave, die]",5,25,0,0,0,0,0,0
4,5,1,I can't breathe! was chosen as the most notabl...,can breathe was chosen as the most notable quo...,"[breathe, choose, notable, quote, year, annual...",21,96,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,3830,0,@banditelli regarding what the PSU president d...,regarding what the psu president does,"[regard, psu, president]",6,32,1,0,0,0,0,0
3813,3831,0,@banditelli But still bothers me that I see no...,but still bothers me that see now follow up re...,"[bother, follow, report]",10,41,1,0,0,0,0,0
3814,3832,0,well now that i've listened to all of into the...,well now that ve listened to all of into the w...,"[ve, listen, wood, listen, fob, nosurprisethere]",18,77,0,1,0,0,0,0
3815,3833,0,Hummingbirds #Are #Experts #at #Hovering #Aft...,hummingbirds are experts at hovering after all...,"[hummingbird, expert, hover, background, motio...",15,81,0,8,1,0,2,0


In [17]:
df_train = df.sample(frac=0.7)
df_validation = df.drop(df_train.index)

In [18]:
df_train

,Tweet index,Label,Tweet text,clean_text,lemmas,word_count,char_count,tag_count,hashtag_count,link_count,smiley_count,mark_count,has_emoji
1225,1226,1,I'm a Victoria Secret model. It's such a secre...,victoria secret model it such secret not even ...,"[victoria, secret, model, secret, victoria, know]",10,51,0,0,1,0,0,0
418,419,1,Left my lunch at home. Swansea canteen outdoin...,left my lunch at home swansea canteen outdoing...,"[leave, lunch, home, swansea, canteen, outdo, ...",18,91,0,0,1,0,0,0
2057,2061,0,@tweetkumud we are here to fight endlessly and...,we are here to fight endlessly and change the ...,"[fight, endlessly, change, system, atleast, ir...",20,95,1,1,0,0,0,0
426,427,1,"love, love, love how the left side of my face ...",love love love how the left side of my face is...,"[love, love, love, left, face, swollen]",12,43,0,0,0,0,0,1
2011,2015,1,"@MattHDGamer FUT14 ofc, the corners <3___<3 th...",fut ofc the corners they where fantastic,"[fut, ofc, corner, fantastic]",7,34,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,1493,1,Welcome to twitter @orafa2 myself n @SlimDandy...,welcome to twitter myself at ur service as ur ...,"[welcome, twitter, ur, service, ur, twitter, c...",11,51,2,0,0,0,0,0
2282,2286,0,95% of my clothes are black. #noshame :bomb:\r,of my clothes are black noshame,"[clothe, black, noshame]",6,26,0,1,0,0,0,1
2368,2372,0,This pressure is unreal. Keep it up\r,this pressure is unreal keep it up,"[pressure, unreal]",7,28,0,0,0,0,0,0
939,940,0,"Wings? I don't have wings! |""Of course not... ...",wings don have wings course not you re boy,"[wing, don, wing, course, boy]",9,34,0,0,0,0,4,0


In [52]:
def new_df_with_all_features(df1, df2):
    cols_to_add = list(set(df2.columns.tolist()) - set(df1.columns.tolist()))
    if 'Tweet index' in cols_to_add:
        cols_to_add.remove('Tweet index')
    new_df = pd.concat((df1.copy(), df2[cols_to_add]), axis=1)
    return new_df

# tweet_text_train = new_df_with_all_features(tweet_text_train, df_train)
# tweet_text_validation = new_df_with_all_features(tweet_validation_train, df_validation)
# tweet_text_test = new_df_with_all_features(tweet_validation_train, df_validation)

In [53]:
# tweet_text_train.head(15)

,Tweet text,length,link_count,exclamation_mark_count,word_count,has_emoji,tag_count,smiley_count,hashtag_count,clean_text,Label
1731,"[hybridcloud, need, hybrid, monitor, monitor, ...",107,1,0,12,0,0,0,1,"needs hybrid monitoring... monitor Azure, plus...",0
2615,"[bet, outside, today, warm, shirt, short, weat...",128,0,0,15,1,1,0,1,"I bet you're gonna be outside today, its soooo...",1
1341,"[ugly, christmas, sweater, woman, sz]",85,2,0,9,0,0,0,0,http://t.co/juRDWnOQ8z so Ugly Christmas Sweat...,0
128,"[sick, look, tweet, oh, yeah, sex, lame]",60,0,0,11,0,0,0,1,Why am I sick? *looks at other tweets* oh yeah...,1
3466,"[mama, korean, award, hold, hong, kong, headli...",103,0,0,21,0,0,0,0,MAMA - a Korean awards show that will be held ...,1
1746,"[bibotour, champwithin, founder, dancer, ricky...",107,0,0,7,0,2,0,4,& founder on w/dancers Ricky & Gracey,0
936,"[world, level]",45,0,0,10,0,0,0,0,The whole world can never get me on my level,1
3150,"[cup, tea, total, ass, switch, coffee, victory...",171,0,0,18,1,1,0,0,"You were my cup of tea, but you were a total a...",0
121,"[mom, picture, grunge, haha, love, momgoal]",140,1,3,20,0,1,0,1,"showed mom this picture of me and & says ""this...",0
3521,"[finnish, sniper, work, fascist, kyiv, nice, guy]",101,0,2,15,0,1,0,1,"NO, there 'only' are some Finnish working with...",1


In [26]:
def join_docs(s):
    '''Joins the strings inside the inner list of a nested list'''
    return ' '.join(s)

df_train['topic_text'] = df_train['lemmas'].apply(join_docs)
df_validation['topic_text'] = df_validation['lemmas'].apply(join_docs)
df_test['topic_text'] = df_test['lemmas'].apply(join_docs)

In [56]:
# tweet_text_train = tweet_text_train[tweet_text_train['Tweet text'] != '']
# tweet_text_validation = tweet_text_validation[tweet_text_validation['Tweet text'] != '']
# tweet_text_test = tweet_text_test[tweet_text_test['Tweet text'] != '']

In [57]:
# tweet_text_train.head(15)

,Tweet text,length,link_count,exclamation_mark_count,word_count,has_emoji,tag_count,smiley_count,hashtag_count,clean_text,Label
1731,hybridcloud need hybrid monitor monitor azure ...,107,1,0,12,0,0,0,1,"needs hybrid monitoring... monitor Azure, plus...",0
2615,bet outside today warm shirt short weather funny,128,0,0,15,1,1,0,1,"I bet you're gonna be outside today, its soooo...",1
1341,ugly christmas sweater woman sz,85,2,0,9,0,0,0,0,http://t.co/juRDWnOQ8z so Ugly Christmas Sweat...,0
128,sick look tweet oh yeah sex lame,60,0,0,11,0,0,0,1,Why am I sick? *looks at other tweets* oh yeah...,1
3466,mama korean award hold hong kong headline amer...,103,0,0,21,0,0,0,0,MAMA - a Korean awards show that will be held ...,1
1746,bibotour champwithin founder dancer ricky grac...,107,0,0,7,0,2,0,4,& founder on w/dancers Ricky & Gracey,0
936,world level,45,0,0,10,0,0,0,0,The whole world can never get me on my level,1
3150,cup tea total ass switch coffee victory_hand,171,0,0,18,1,1,0,0,"You were my cup of tea, but you were a total a...",0
121,mom picture grunge haha love momgoal,140,1,3,20,0,1,0,1,"showed mom this picture of me and & says ""this...",0
3521,finnish sniper work fascist kyiv nice guy,101,0,2,15,0,1,0,1,"NO, there 'only' are some Finnish working with...",1


In [58]:
tweet_text_train.shape

(2655, 11)

## Topic modeling baselines

In [19]:
count_vectorizer = CountVectorizer(
    analyzer='word',
    min_df=20,
    max_df=0.5,
)

tfidf_vectorizer = TfidfVectorizer(
    analyzer='word',
    min_df=20,
    max_df=0.5,
)

tweet_text_count_train = count_vectorizer.fit_transform(df_train['clean_text'])
tweet_text_count_validation = count_vectorizer.transform(df_validation['clean_text'])
tweet_text_count_test = count_vectorizer.transform(df_test['clean_text'])

tweet_text_tfidf_train = tfidf_vectorizer.fit_transform(df_train['clean_text'])
tweet_text_tfidf_validation = tfidf_vectorizer.transform(df_validation['clean_text'])
tweet_text_tfidf_test = tfidf_vectorizer.transform(df_test['clean_text'])

In [ ]:
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

inertia = []
range_ = list(range(2, 60))
for i in range_:
    model = KMeans(i)
    model.fit(tweet_text_count_train)
    inertia.append(model.inertia_)
    
plt.plot(range_, inertia)
plt.xlabel('Number of components')
plt.ylabel('Inertia')
plt.show()

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_fast._relocate_empty_clusters_sparse'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt: 


In [ ]:
model = KMeans(26)
model.fit(tweet_text_count_train)

kmeans_count_labels_train = model.predict(tweet_text_count_train)
kmeans_count_labels_validation = model.predict(tweet_text_count_validation)
kmeans_count_labels_test = model.predict(tweet_text_count_test)

In [ ]:
inertia = []
for i in range_:
    model = KMeans(i)
    model.fit(tweet_text_tfidf_train)
    inertia.append(model.inertia_)
plt.plot(range_, inertia)
plt.xlabel('Number of components')
plt.ylabel('Inertia')
plt.show()

In [ ]:
model = KMeans(17)
model.fit(tweet_text_count_train)

kmeans_tfidf_labels_train = model.predict(tweet_text_count_train)
kmeans_tfidf_labels_validation = model.predict(tweet_text_count_validation)
kmeans_tfidf_labels_test = model.predict(tweet_text_count_test)

In [563]:
#!pip install bertopic

  Using cached bertopic-0.10.0-py2.py3-none-any.whl (58 kB)
  Using cached plotly-5.8.0-py2.py3-none-any.whl (15.2 MB)
  Using cached PyYAML-5.4.1-cp39-cp39-win_amd64.whl (213 kB)
  Using cached hdbscan-0.8.28.tar.gz (5.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached umap_learn-0.5.3-py3-none-any.whl
  Using cached sentence_transformers-2.2.0-py3-none-any.whl
  Using cached tenacity-8.0.1-py3-none-any.whl (24 kB)
  Using cached sentencepiece-0.1.96-cp39-cp39-win_amd64.whl (1.1 MB)
  Using cached torchvision-0.12.0-cp39-cp39-win_amd64.whl (1.0 MB)
  Using cached pynndescent-0.5.7-py3-none-any.whl
  Using cached numpy-1.21.6-cp39-cp39-win_amd64.whl (14.0 MB)
Failed to build hdbscan


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [40 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-39
  creating build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\flat.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\hdbscan_.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\plots.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\prediction.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\robust_single_linkage_.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\validity.py -> build\lib.win-amd64-cpython-39\hdbscan
  copying hdbscan\__init__.py -> build\lib.win-amd64-cpython-39\hdbscan
  creating build\lib.win-amd64-cpython-39\hdbscan\tests
  copying hdbscan\tests\test_flat.py -> build\lib.win-amd64-cpython-39\hdbscan\tests
  copying hdbscan\tests\test_hdbscan.py -> build\lib.win-amd64-cpython-39\hdbscan\tests
  copying 

In [28]:
df_train.reset_index(drop=True, inplace=True)
df_validation.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

### BERTopic

In [88]:
from bertopic import BERTopic

topic_model = BERTopic(min_topic_size=50)
topics, probs = topic_model.fit_transform(df_train['topic_text'])

pred_train = topic_model.transform(df_train['topic_text'])
pred_validation = topic_model.transform(df_validation['topic_text'])
pred_test = topic_model.transform(df_test['topic_text'])

In [89]:
topic_model.get_topic(0)

[('christmas', 0.12289697955789392),
 ('look', 0.03731473284405396),
 ('gift', 0.03379362592458907),
 ('girl', 0.030105766651632084),
 ('song', 0.029487034269259093),
 ('wear', 0.028594606551575374),
 ('day', 0.026087743525930657),
 ('hair', 0.02563227804746289),
 ('selfie', 0.025490263223589306),
 ('music', 0.025293063208118933)]

In [90]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1573,-1_love_like_year_time
1,0,274,0_christmas_look_gift_girl
2,1,177,1_police_white_racism_rape
3,2,164,2_game_play_win_good
4,3,116,3_day_week_today_work
5,4,114,4_sleep_wake_morning_hour
6,5,93,5_love_funny_dad_child
7,6,89,6_school_paper_final_study
8,7,72,7_tweet_twitter_follow_facebook


In [91]:
df_train['topic'] = pred_train[0]
df_validation['topic'] = pred_validation[0]
df_test['topic'] = pred_test[0]

In [92]:
df_train

,Tweet index,Label,Tweet text,clean_text,lemmas,word_count,char_count,tag_count,hashtag_count,link_count,...,17_topic,18_topic,19_topic,20_topic,21_topic,22_topic,23_topic,24_topic,25_topic,26_topic
0,1226,1,I'm a Victoria Secret model. It's such a secre...,victoria secret model it such secret not even ...,"[victoria, secret, model, secret, victoria, know]",10,51,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,419,1,Left my lunch at home. Swansea canteen outdoin...,left my lunch at home swansea canteen outdoing...,"[leave, lunch, home, swansea, canteen, outdo, ...",18,91,0,0,1,...,1,0,0,0,0,0,0,0,0,0
2,2061,0,@tweetkumud we are here to fight endlessly and...,we are here to fight endlessly and change the ...,"[fight, endlessly, change, system, atleast, ir...",20,95,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,427,1,"love, love, love how the left side of my face ...",love love love how the left side of my face is...,"[love, love, love, left, face, swollen]",12,43,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2015,1,"@MattHDGamer FUT14 ofc, the corners <3___<3 th...",fut ofc the corners they where fantastic,"[fut, ofc, corner, fantastic]",7,34,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,1493,1,Welcome to twitter @orafa2 myself n @SlimDandy...,welcome to twitter myself at ur service as ur ...,"[welcome, twitter, ur, service, ur, twitter, c...",11,51,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2668,2286,0,95% of my clothes are black. #noshame :bomb:\r,of my clothes are black noshame,"[clothe, black, noshame]",6,26,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2669,2372,0,This pressure is unreal. Keep it up\r,this pressure is unreal keep it up,"[pressure, unreal]",7,28,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2670,940,0,"Wings? I don't have wings! |""Of course not... ...",wings don have wings course not you re boy,"[wing, don, wing, course, boy]",9,34,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Embedding

In [54]:
from podium import Vocab, Field, LabelField
from podium.datasets import TabularDataset
from podium.vectorizers import GloVe

In [47]:
tweet_embedding_train = df_train[['topic_text', 'Label']]
tweet_embedding_validation = df_validation[['topic_text', 'Label']]
tweet_embedding_test = df_test[['topic_text', 'Label']]

In [48]:
tweet_embedding_train.reset_index(drop=True, inplace=True)
tweet_embedding_validation.reset_index(drop=True, inplace=True)
tweet_embedding_test.reset_index(drop=True, inplace=True)

In [55]:
max_vocab_size = 10_000
vocab = Vocab(max_size=max_vocab_size, min_freq=20)

TWEET = Field('text', numericalizer=vocab)
LABEL = LabelField('Label')

fields = [TWEET, LABEL]

train = TabularDataset.from_pandas(df_train[['topic_text', 'Label']], fields)
validation = TabularDataset.from_pandas(df_validation[['topic_text', 'Label']], fields)
test = TabularDataset.from_pandas(df_test[['topic_text', 'Label']], fields)
train.finalize_fields()

glove = GloVe()
embeddings = glove.load_vocab(vocab)

train_batch = train.batch(add_padding=True)
validation_batch = validation.batch(add_padding=True)
test_batch = test.batch(add_padding=True)

In [56]:
train_batch['text'].astype(int)[0:5]

array([[ 0,  0,  0,  0,  0, 11,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1],
       [67,  0, 89,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1],
       [ 0,  0, 79,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1],
       [ 2,  2,  2,  0, 80,  0,  1,  1,  1,  1,  

In [57]:
tweet_train = embeddings[train_batch['text'].astype(int)]
tweet_validation = embeddings[validation_batch['text'].astype(int)]
tweet_test = embeddings[test_batch['text'].astype(int)]

# Mean
tweet_train_mean = tweet_train.mean(axis=1)
tweet_validation_mean = tweet_validation.mean(axis=1)
tweet_test_mean = tweet_test.mean(axis=1)

In [58]:
embeddings_train_mean_df = pd.DataFrame(tweet_train_mean)
df_train = pd.merge(df_train, embeddings_train_mean_df, left_index=True, right_index=True)

embeddings_validation_mean_df = pd.DataFrame(tweet_validation_mean)
df_validation = pd.merge(df_validation, embeddings_validation_mean_df, left_index=True, right_index=True)

embeddings_test_mean_df = pd.DataFrame(tweet_test_mean)
df_test = pd.merge(df_test, embeddings_test_mean_df, left_index=True, right_index=True)

In [59]:
df_train

,Tweet index,Label,Tweet text,clean_text,lemmas,word_count,char_count,tag_count,hashtag_count,link_count,...,290,291,292,293,294,295,296,297,298,299
0,1226,1,I'm a Victoria Secret model. It's such a secre...,victoria secret model it such secret not even ...,"[victoria, secret, model, secret, victoria, know]",10,51,0,0,1,...,-0.734994,-0.656253,-1.247315,-0.715902,-0.472768,0.245063,-1.055144,-0.341575,-1.258775,-0.501747
1,419,1,Left my lunch at home. Swansea canteen outdoin...,left my lunch at home swansea canteen outdoing...,"[leave, lunch, home, swansea, canteen, outdo, ...",18,91,0,0,1,...,-0.743841,-0.647377,-1.135379,-0.688410,-0.585451,0.292726,-0.978633,-0.313849,-1.094880,-0.424406
2,2061,0,@tweetkumud we are here to fight endlessly and...,we are here to fight endlessly and change the ...,"[fight, endlessly, change, system, atleast, ir...",20,95,1,1,0,...,-0.748988,-0.654107,-1.158139,-0.682845,-0.588523,0.297747,-1.003470,-0.309303,-1.111962,-0.444621
3,427,1,"love, love, love how the left side of my face ...",love love love how the left side of my face is...,"[love, love, love, left, face, swollen]",12,43,0,0,0,...,-0.716877,-0.623747,-1.275515,-0.717432,-0.334869,0.194307,-1.042801,-0.342240,-1.329015,-0.548489
4,2015,1,"@MattHDGamer FUT14 ofc, the corners <3___<3 th...",fut ofc the corners they where fantastic,"[fut, ofc, corner, fantastic]",7,34,1,0,0,...,-0.745567,-0.661025,-1.299867,-0.733037,-0.440542,0.246756,-1.088922,-0.354721,-1.329997,-0.539491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,1493,1,Welcome to twitter @orafa2 myself n @SlimDandy...,welcome to twitter myself at ur service as ur ...,"[welcome, twitter, ur, service, ur, twitter, c...",11,51,2,0,0,...,-0.703266,-0.638952,-1.253360,-0.704631,-0.435985,0.229188,-1.043825,-0.335427,-1.280024,-0.526104
2668,2286,0,95% of my clothes are black. #noshame :bomb:\r,of my clothes are black noshame,"[clothe, black, noshame]",6,26,0,1,0,...,-0.741880,-0.660962,-1.329347,-0.741173,-0.402545,0.233630,-1.107008,-0.364313,-1.379796,-0.560553
2669,2372,0,This pressure is unreal. Keep it up\r,this pressure is unreal keep it up,"[pressure, unreal]",7,28,0,0,0,...,-0.738192,-0.660898,-1.358828,-0.749309,-0.364549,0.220503,-1.125094,-0.373904,-1.429596,-0.581615
2670,940,0,"Wings? I don't have wings! |""Of course not... ...",wings don have wings course not you re boy,"[wing, don, wing, course, boy]",9,34,0,0,0,...,-0.739118,-0.657139,-1.282313,-0.720612,-0.445044,0.245276,-1.075079,-0.345795,-1.310360,-0.521672


In [ ]:
## Features
## Broj neg rijeci
## Broj poz rijeci
## Omjer
## Udaljenost izmedu poz i neg rijeci

Potrebno dodatno preprocesat da samo rjeci ostanu

In [60]:
import nltk
nltk.download('vader_lexicon') # if error run this

[nltk_data] Downloading package vader_lexicon to C:\Users\Marino
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [63]:
from nltk.sentiment import SentimentIntensityAnalyzer
import copy

def pos_neg_words(df, limit):
    sid = SentimentIntensityAnalyzer()

    neg_words = []
    neg_word_count = []
    pos_words = []
    pos_word_count = []
    for index, row in df.iterrows():
        lemmas = []
        if len(row['topic_text']) > 0:
            doc = nlp(row['topic_text'])
            for token in doc:
                lemmas.append(token.lemma_)

            current_pos = []
            current_neut = []
            current_neg = []
            for word in lemmas:
                if (sid.polarity_scores(word)['compound']) >= limit:
                    current_pos.append(word)
                elif (sid.polarity_scores(word)['compound']) <= -limit:
                    current_neg.append(word)
                else:
                    current_neut.append(word)

            neg_words.append(copy.copy(current_neg))
            neg_word_count.append(copy.copy(len(current_neg)))
            pos_words.append(copy.copy(current_pos))
            pos_word_count.append(copy.copy(len(current_pos)))
        else:
            neg_words.append([])
            neg_word_count.append(0)
            pos_words.append([])
            pos_word_count.append(0)
    return neg_words, neg_word_count, pos_words, pos_word_count

In [62]:
df_train

,Tweet index,Label,Tweet text,clean_text,lemmas,word_count,char_count,tag_count,hashtag_count,link_count,...,290,291,292,293,294,295,296,297,298,299
0,1226,1,I'm a Victoria Secret model. It's such a secre...,victoria secret model it such secret not even ...,"[victoria, secret, model, secret, victoria, know]",10,51,0,0,1,...,-0.734994,-0.656253,-1.247315,-0.715902,-0.472768,0.245063,-1.055144,-0.341575,-1.258775,-0.501747
1,419,1,Left my lunch at home. Swansea canteen outdoin...,left my lunch at home swansea canteen outdoing...,"[leave, lunch, home, swansea, canteen, outdo, ...",18,91,0,0,1,...,-0.743841,-0.647377,-1.135379,-0.688410,-0.585451,0.292726,-0.978633,-0.313849,-1.094880,-0.424406
2,2061,0,@tweetkumud we are here to fight endlessly and...,we are here to fight endlessly and change the ...,"[fight, endlessly, change, system, atleast, ir...",20,95,1,1,0,...,-0.748988,-0.654107,-1.158139,-0.682845,-0.588523,0.297747,-1.003470,-0.309303,-1.111962,-0.444621
3,427,1,"love, love, love how the left side of my face ...",love love love how the left side of my face is...,"[love, love, love, left, face, swollen]",12,43,0,0,0,...,-0.716877,-0.623747,-1.275515,-0.717432,-0.334869,0.194307,-1.042801,-0.342240,-1.329015,-0.548489
4,2015,1,"@MattHDGamer FUT14 ofc, the corners <3___<3 th...",fut ofc the corners they where fantastic,"[fut, ofc, corner, fantastic]",7,34,1,0,0,...,-0.745567,-0.661025,-1.299867,-0.733037,-0.440542,0.246756,-1.088922,-0.354721,-1.329997,-0.539491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,1493,1,Welcome to twitter @orafa2 myself n @SlimDandy...,welcome to twitter myself at ur service as ur ...,"[welcome, twitter, ur, service, ur, twitter, c...",11,51,2,0,0,...,-0.703266,-0.638952,-1.253360,-0.704631,-0.435985,0.229188,-1.043825,-0.335427,-1.280024,-0.526104
2668,2286,0,95% of my clothes are black. #noshame :bomb:\r,of my clothes are black noshame,"[clothe, black, noshame]",6,26,0,1,0,...,-0.741880,-0.660962,-1.329347,-0.741173,-0.402545,0.233630,-1.107008,-0.364313,-1.379796,-0.560553
2669,2372,0,This pressure is unreal. Keep it up\r,this pressure is unreal keep it up,"[pressure, unreal]",7,28,0,0,0,...,-0.738192,-0.660898,-1.358828,-0.749309,-0.364549,0.220503,-1.125094,-0.373904,-1.429596,-0.581615
2670,940,0,"Wings? I don't have wings! |""Of course not... ...",wings don have wings course not you re boy,"[wing, don, wing, course, boy]",9,34,0,0,0,...,-0.739118,-0.657139,-1.282313,-0.720612,-0.445044,0.245276,-1.075079,-0.345795,-1.310360,-0.521672


In [68]:
neg_words_train, neg_word_count_train, pos_words_train, pos_word_count_train = pos_neg_words(df_train, 0.2)
neg_words_val, neg_word_count_val, pos_words_val, pos_word_count_val = pos_neg_words(df_validation, 0.2)
neg_words_test, neg_word_count_text, pos_words_test, pos_word_count_test = pos_neg_words(df_test, 0.2)

In [65]:
df_train['neg_word_count'] = neg_word_count_train
#df_train['neg_word'] = neg_words_train
df_train['pos_word_count'] = pos_word_count_train
#df_train['pos_word'] = pos_words_train
df_train

,Tweet index,Label,Tweet text,clean_text,lemmas,word_count,char_count,tag_count,hashtag_count,link_count,...,292,293,294,295,296,297,298,299,neg_word_count,pos_word_count
0,1226,1,I'm a Victoria Secret model. It's such a secre...,victoria secret model it such secret not even ...,"[victoria, secret, model, secret, victoria, know]",10,51,0,0,1,...,-1.247315,-0.715902,-0.472768,0.245063,-1.055144,-0.341575,-1.258775,-0.501747,0,0
1,419,1,Left my lunch at home. Swansea canteen outdoin...,left my lunch at home swansea canteen outdoing...,"[leave, lunch, home, swansea, canteen, outdo, ...",18,91,0,0,1,...,-1.135379,-0.688410,-0.585451,0.292726,-0.978633,-0.313849,-1.094880,-0.424406,0,1
2,2061,0,@tweetkumud we are here to fight endlessly and...,we are here to fight endlessly and change the ...,"[fight, endlessly, change, system, atleast, ir...",20,95,1,1,0,...,-1.158139,-0.682845,-0.588523,0.297747,-1.003470,-0.309303,-1.111962,-0.444621,1,0
3,427,1,"love, love, love how the left side of my face ...",love love love how the left side of my face is...,"[love, love, love, left, face, swollen]",12,43,0,0,0,...,-1.275515,-0.717432,-0.334869,0.194307,-1.042801,-0.342240,-1.329015,-0.548489,0,3
4,2015,1,"@MattHDGamer FUT14 ofc, the corners <3___<3 th...",fut ofc the corners they where fantastic,"[fut, ofc, corner, fantastic]",7,34,1,0,0,...,-1.299867,-0.733037,-0.440542,0.246756,-1.088922,-0.354721,-1.329997,-0.539491,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,1493,1,Welcome to twitter @orafa2 myself n @SlimDandy...,welcome to twitter myself at ur service as ur ...,"[welcome, twitter, ur, service, ur, twitter, c...",11,51,2,0,0,...,-1.253360,-0.704631,-0.435985,0.229188,-1.043825,-0.335427,-1.280024,-0.526104,0,1
2668,2286,0,95% of my clothes are black. #noshame :bomb:\r,of my clothes are black noshame,"[clothe, black, noshame]",6,26,0,1,0,...,-1.329347,-0.741173,-0.402545,0.233630,-1.107008,-0.364313,-1.379796,-0.560553,0,0
2669,2372,0,This pressure is unreal. Keep it up\r,this pressure is unreal keep it up,"[pressure, unreal]",7,28,0,0,0,...,-1.358828,-0.749309,-0.364549,0.220503,-1.125094,-0.373904,-1.429596,-0.581615,1,0
2670,940,0,"Wings? I don't have wings! |""Of course not... ...",wings don have wings course not you re boy,"[wing, don, wing, course, boy]",9,34,0,0,0,...,-1.282313,-0.720612,-0.445044,0.245276,-1.075079,-0.345795,-1.310360,-0.521672,0,0


In [69]:
df_validation['neg_word_count'] = neg_word_count_val
df_validation['pos_word_count'] = pos_word_count_val
df_test['neg_word_count'] = neg_word_count_text
df_test['pos_word_count'] = pos_word_count_test

In [97]:
topic_dummies = []
for i in range(9):
    topic_dummies.append(str(i-1)+'_topic')
topic_dummies

['-1_topic',
 '0_topic',
 '1_topic',
 '2_topic',
 '3_topic',
 '4_topic',
 '5_topic',
 '6_topic',
 '7_topic']

In [98]:
df_train[topic_dummies] = pd.get_dummies(df_train['topic'])
df_validation[topic_dummies] = pd.get_dummies(df_validation['topic'])
df_test[topic_dummies] = pd.get_dummies(df_test['topic'])
df_train

,Tweet index,Label,Tweet text,clean_text,lemmas,word_count,char_count,tag_count,hashtag_count,link_count,...,pos_word_count,-1_topic,0_topic,1_topic,2_topic,3_topic,4_topic,5_topic,6_topic,7_topic
0,1226,1,I'm a Victoria Secret model. It's such a secre...,victoria secret model it such secret not even ...,"[victoria, secret, model, secret, victoria, know]",10,51,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,419,1,Left my lunch at home. Swansea canteen outdoin...,left my lunch at home swansea canteen outdoing...,"[leave, lunch, home, swansea, canteen, outdo, ...",18,91,0,0,1,...,1,1,0,0,0,0,0,0,0,0
2,2061,0,@tweetkumud we are here to fight endlessly and...,we are here to fight endlessly and change the ...,"[fight, endlessly, change, system, atleast, ir...",20,95,1,1,0,...,0,1,0,0,0,0,0,0,0,0
3,427,1,"love, love, love how the left side of my face ...",love love love how the left side of my face is...,"[love, love, love, left, face, swollen]",12,43,0,0,0,...,3,1,0,0,0,0,0,0,0,0
4,2015,1,"@MattHDGamer FUT14 ofc, the corners <3___<3 th...",fut ofc the corners they where fantastic,"[fut, ofc, corner, fantastic]",7,34,1,0,0,...,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2667,1493,1,Welcome to twitter @orafa2 myself n @SlimDandy...,welcome to twitter myself at ur service as ur ...,"[welcome, twitter, ur, service, ur, twitter, c...",11,51,2,0,0,...,1,0,0,0,0,0,0,0,0,1
2668,2286,0,95% of my clothes are black. #noshame :bomb:\r,of my clothes are black noshame,"[clothe, black, noshame]",6,26,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2669,2372,0,This pressure is unreal. Keep it up\r,this pressure is unreal keep it up,"[pressure, unreal]",7,28,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2670,940,0,"Wings? I don't have wings! |""Of course not... ...",wings don have wings course not you re boy,"[wing, don, wing, course, boy]",9,34,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [100]:
from sklearn.linear_model import LogisticRegression
X_ = df_train[['word_count', 'char_count', 'tag_count', 'hashtag_count', 'link_count', 'smiley_count', 'mark_count', 'has_emoji', 'neg_word_count', 'pos_word_count',
               '-1_topic', '0_topic', '1_topic', '2_topic', '3_topic', '4_topic', '5_topic', '6_topic', '7_topic']]
y_ = df_train['Label']
clf = LogisticRegression(random_state=0, solver='liblinear').fit(X_, y_)
print('Train score')
print(clf.score(X_, y_))
print('Validation score')
X_val = df_validation[['word_count', 'char_count', 'tag_count', 'hashtag_count', 'link_count', 'smiley_count', 'mark_count', 'has_emoji', 'neg_word_count', 'pos_word_count',
               '-1_topic', '0_topic', '1_topic', '2_topic', '3_topic', '4_topic', '5_topic', '6_topic', '7_topic']]
y_val = df_validation['Label']
print(clf.score(X_val, y_val))
print('Test score')
X_val = df_test[['word_count', 'char_count', 'tag_count', 'hashtag_count', 'link_count', 'smiley_count', 'mark_count', 'has_emoji', 'neg_word_count', 'pos_word_count',
               '-1_topic', '0_topic', '1_topic', '2_topic', '3_topic', '4_topic', '5_topic', '6_topic', '7_topic']]
y_val = df_test['Label']
print(clf.score(X_val, y_val))

Train score
0.6347305389221557
Validation score
0.611353711790393
Test score
0.6033163265306123


In [ ]:
## Pos and neg words within 4 words
df_train[['clean_text', 'pos_word', 'pos_word_count', 'neg_word', 'neg_word_count']].iloc[25]

In [ ]:
def pos_neg_within_n(df, n=4):

    ret_array = []

    for index, row in df.iterrows():
        if row['pos_word_count'] > 0 and row['neg_word_count'] > 0:
            doc = nlp(row['clean_text'])
            lemmas = []
            for token in doc:
                lemmas.append(token.lemma_)

            pos_indexes = np.array([])
            for word in row['pos_word']:
                pos_indexes = np.append(pos_indexes, np.where(np.array(lemmas) == word))
            neg_indexes = np.array([])
            for word in row['neg_word']:
                neg_indexes = np.append(neg_indexes, np.where(np.array(lemmas) == word))

            bool_val = 0
            for idx in pos_indexes:
                if (abs(neg_indexes-idx) < n).any():
                    bool_val = 1
            ret_array.append(copy.copy(bool_val))
        else:
            ret_array.append(0)
    return ret_array

In [ ]:
within_5_train = pos_neg_within_n(df_train, n=5)
within_5_val = pos_neg_within_n(df_validation, n=5)

In [ ]:
df_train['pos_neg_within_5'] = within_5_train
df_validation['pos_neg_within_5'] = within_5_val
df_train

In [ ]:
X_ = df_train[[ 'pos_neg_within_5']]
y_ = df_train['Label']
clf = LogisticRegression().fit(X_, y_)
print('Train score')
print(clf.score(X_, y_))
print('Validation score')
X_val = df_validation[[ 'pos_neg_within_5']]
y_val = df_validation['Label']
print(clf.score(X_val, y_val))

In [ ]:
within_5_val

### Grid search

1. treba koristit tweet_text_train, tweet_text_validation i tweet_text_test s kombiniranim znacajkama iz df_train, df_validation, df_test, a ne da samo koristi embeddinge - ako ne radi bolje tako onda cemo odjebat embeddinge
2. grid mora koristit minimalno feature selekciju, skaliranje i model (za dtc ne treba skalirat, lr moze i polynomial transform)
3. pazi na data leakove, nemres fitat search i onda evaluirat na njemu test, treba natrenirat novi model s tim najboljim hiperparametrima na trainu i onda predictat na testu
4. povecat broj parametara, ovo je jako slabo, grid se vrti po 30ak sekundi

In [607]:
X_ = df_train.drop(columns=['Tweet index', 'Label', 'Tweet text', 'clean_text'], axis=1)
y_ = df_train['Label']

X_val = df_validation.drop(['Tweet index', 'Label', 'Tweet text', 'clean_text'], axis=1)
y_val = df_validation['Label']

X_test = df_test.drop(['Tweet index', 'Label', 'Tweet text'], axis=1)
y_test = df_test['Label']

In [618]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import classification_report

pipeline_lr = Pipeline([
    ('selection', SelectKBest()), 
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression())
])

params = {
    'selection__k': list(range(2, 20, 3)),
    'lr__penalty': ['l1', 'l2', 'elasticnet'], 
    'lr__C': np.logspace(-4, 4, 20), 
    'lr__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 
}


search = GridSearchCV(pipeline_lr, param_grid=params, cv=5)
search.fit(X_, y_)

print(f'Best score: {search.best_score_}')
print(f'Best score: {search.best_params_}')

Best score: 0.6051566383142568
Best score: {'lr__C': 0.08858667904100823, 'lr__penalty': 'l2', 'lr__solver': 'newton-cg', 'selection__k': 8}


In [617]:
pipeline_lr_best = Pipeline([
    ('selection', SelectKBest(k=8)), 
    ('scaler', StandardScaler()),
    ('lr', LogisticRegression(C=0.0885, penalty='l2', solver='newton-cg'))
])

pipeline_lr_best.fit(X_, y_)
y_train_pred = pipeline_lr_best.predict(X_)
print(classification_report(y_train_pred, y_))

y_validation_pred = pipeline_lr_best.predict(X_val)
print(classification_report(y_validation_pred, y_val))

              precision    recall  f1-score   support

           0       0.51      0.64      0.57      1073
           1       0.71      0.58      0.64      1599

    accuracy                           0.60      2672
   macro avg       0.61      0.61      0.60      2672
weighted avg       0.63      0.60      0.61      2672

              precision    recall  f1-score   support

           0       0.53      0.63      0.58       467
           1       0.71      0.62      0.66       678

    accuracy                           0.62      1145
   macro avg       0.62      0.63      0.62      1145
weighted avg       0.64      0.62      0.63      1145



In [ ]:
from sklearn.tree import DecisionTreeClassifier

params = {'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 6, 7, 8, 9],
              'criterion' : ['gini', 'entropy', 'log_loss']
             }


X_ = df_train.drop(['Tweet index', 'Label', 'Tweet text', 'clean_text', 'pos_word', 'neg_word'], axis=1)
y_ = df_train['Label']
clf = DecisionTreeClassifier()

model = GridSearchCV(clf, param_grid=params, cv=5)

model.fit(X_, y_)

print('Train score')
print(model.score(X_, y_))
print('Validation score')
X_val = df_validation.drop(['Tweet index', 'Label', 'Tweet text', 'clean_text', 'pos_word', 'neg_word'], axis=1)
y_val = df_validation['Label']
print(model.score(X_val, y_val))